In [1]:
# !pip install azure-cognitiveservices-vision-customvision
# !pip install azure-cognitiveservices-speech
# !pip install msrest
# !pip install openai
# !pip install gradio
# !pip install opencv-python
# !pip install pillow

In [ ]:
import gradio as gr
import os
import io
import cv2
import unicodedata
from PIL import Image
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from msrest.authentication import ApiKeyCredentials
from openai import AzureOpenAI

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# Azure CustomVision 초기화
CUSTOMVISION_ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=CUSTOMVISION_ENDPOINT, credentials=credentials)

################# 자음/모음 분리 함수 #################
def split_hangul(word):
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    separated = []
    for char in word:
        if '가' <= char <= '힣':
            code = ord(char) - ord('가')
            initial = code // (21 * 28)
            medial = (code % (21 * 28)) // 28
            final = code % 28
            separated.append(initial_consonants[initial])
            separated.append(medial_vowels[medial])
            if final_consonants[final]:
                separated.append(final_consonants[final])
        else:
            separated.append(char)
    return separated

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
               "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
               "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
               "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}
reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def display_images(word):
    separated_word = split_hangul(word)
    result = [reverse_hangul_dict.get(jamo, "") for jamo in separated_word if jamo in reverse_hangul_dict]

    image_folder = "Images/Basic"  # 이미지 폴더 경로
    image_paths = []

    for tag in result:
        image_path = os.path.join(image_folder, f"{tag}.jpg")
        if os.path.exists(image_path):
            image_paths.append(image_path)

    return image_paths

def generate_gallery(word):
    images = display_images(word)
    return images


################# CUSTOM VISION #################
def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()

    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)

        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img,
                          (left, top),
                          (left + width, top + height),
                          color,
                          2)

            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img,
                        label,
                        (left, top - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        color,
                        2)

    return img

def process_frame(frame):
    if frame is None:
        return None

    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)

    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)

        # Draw boxes on frame
        annotated_frame = draw_boxes(frame, results.predictions)

        return annotated_frame

    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame



################# CHATBOT #################
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정
        
        # 검색 결과 없는 기본 메시지 처리
        if "요청된 정보는 검색된 데이터에서 찾을 수 없습니다. 다른 쿼리나 주제를 시도해 보세요." in assistant_reply:
            assistant_reply = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        try:
            citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
            video_url = citations[-2]
            image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음
        except (KeyError, IndexError):
            # 출처 정보를 찾을 수 없을 경우 예외 처리
            video_url = None
            image_urls = []

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history, video_url, image_urls

    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))
        return chat_history, chat_history, None, []


# Gradio 인터페이스
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("## Sign Language Chatbot")
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("★ Clear Chat ★")
            with gr.Row():
                video_display = gr.Video(label="수어 영상")
                image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column():
            toggle_button = gr.Button("지문자 연습하기!")

            # 버튼 클릭으로 표시/숨기기되는 영역
            with gr.Column(visible=False) as customvision_section:
                gr.Markdown("## 실시간 CustomVision 객체 인식")
                practice_input = gr.Textbox(label="단어 입력", placeholder="단어를 입력하세요")
                practice_gallery = gr.Gallery(label="결과 이미지", columns=6, rows=1)
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")

                practice_input.change(generate_gallery, inputs=practice_input, outputs=practice_gallery)


    # visible 상태를 관리하기 위한 State
    visible_state = gr.State(value=True)

    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state, video_display, image_display])
    # Clear 버튼 이벤트 연결
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )

    # 버튼 클릭 이벤트로 영역 토글
    toggle_button.click(
        lambda visible: (not visible, gr.update(visible=not visible)),  # 현재 visible 상태를 반전
        inputs=[visible_state],  # 현재 visible 상태를 입력으로 받음
        outputs=[visible_state, customvision_section],  # 상태와 섹션의 visible 업데이트
    )

    # Custom Vision 웹캠 스트림 설정
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=webcam_output)

demo.launch()

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.
